In [1]:
# As usual, importing the libraries we need
import json
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import pearsonr, spearmanr
from statsmodels.stats.multitest import multipletests

In [3]:
# Clean the corona data
corona_df = pd.read_csv("../fyp2022p0105/data/raw/corona/dk_corona.csv", sep = "\t")

with open("../fyp2022p0105/data/raw/metadata/dk_metadata.json", 'r') as f:
   country_metadata = json.load(f)

region_map = {country_metadata["country_metadata"][i]["covid_region_code"]: country_metadata["country_metadata"][i]["iso3166-2_code"] for i in range(len(country_metadata["country_metadata"]))}
corona_df["region"] = corona_df["region_code"].map(region_map)

corona_df

,date,region_code,hospitalized_addition,region
0,2020-03-01,Capital Region of Denmark,1,DK-84
1,2020-03-02,Capital Region of Denmark,0,DK-84
2,2020-03-03,Capital Region of Denmark,1,DK-84
3,2020-03-04,Capital Region of Denmark,0,DK-84
4,2020-03-05,Capital Region of Denmark,1,DK-84
...,...,...,...,...
1755,2021-02-11,North Denmark Region,1,DK-81
1756,2021-02-12,North Denmark Region,1,DK-81
1757,2021-02-13,North Denmark Region,1,DK-81
1758,2021-02-14,North Denmark Region,1,DK-81


In [4]:
# Clean the weather data
weather_df = pd.read_csv("../fyp2022p0105/data/raw/weather/weather.csv")

weather_df["TemperatureAboveGround"] = weather_df["TemperatureAboveGround"] - 273.15
weather_df = weather_df[weather_df["iso3166-2"].str.startswith("DK")]

weather_df

,date,iso3166-2,RelativeHumiditySurface,SolarRadiation,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed
16,2020-02-13,DK-81,86.342782,2.555655e+06,2.403426e+06,3.145413,0.000464,2.071701,3.384652
17,2020-02-13,DK-82,87.751343,2.153149e+06,2.399142e+06,3.577344,0.001325,2.990025,3.801099
18,2020-02-13,DK-83,86.736915,1.451333e+06,2.402263e+06,3.724953,0.003497,3.131156,4.773278
19,2020-02-13,DK-84,84.394247,3.058965e+06,2.409240e+06,3.970543,0.001290,2.842956,4.884642
20,2020-02-13,DK-85,86.200033,2.891117e+06,2.408930e+06,3.763832,0.002166,3.071611,5.276005
...,...,...,...,...,...,...,...,...,...
19888,2021-02-15,DK-81,76.951013,3.486211e+04,2.452897e+06,-0.972067,0.002582,0.015256,5.553239
19889,2021-02-15,DK-82,72.866680,3.564665e+04,2.447235e+06,-0.881744,0.002975,0.076808,4.863227
19890,2021-02-15,DK-83,70.380505,6.083510e+04,2.450499e+06,-0.896892,0.004327,1.938972,5.361945
19891,2021-02-15,DK-84,78.648719,4.083331e+05,2.464868e+06,-1.458766,0.002089,1.318707,4.277723


In [5]:
# Merging corona and weather data
df = corona_df.merge(weather_df, left_on = ["date", "region"], right_on = ["date", "iso3166-2"])
df = df.drop(["region_code", "region"], axis = 1)

# Lost because the weather data does not cover some corona data
print(corona_df.shape[0] - df.shape[0])
# Lost because the corona data starts in Mar 1st, 2020
print(weather_df.shape[0] - df.shape[0])

df

0
85


,date,hospitalized_addition,iso3166-2,RelativeHumiditySurface,SolarRadiation,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed
0,2020-03-01,1,DK-84,79.371362,3.383109e+06,2.370635e+06,5.064128,0.000764,2.595843,6.668466
1,2020-03-02,0,DK-84,86.574612,3.303007e+06,2.380293e+06,4.470362,0.001416,4.286374,2.475038
2,2020-03-03,1,DK-84,93.285949,9.690623e+04,2.395165e+06,3.884757,0.002084,1.676674,2.345198
3,2020-03-04,0,DK-84,86.105840,3.227602e+06,2.407377e+06,4.677848,0.000926,4.771363,4.631544
4,2020-03-05,1,DK-84,86.688654,2.998848e+06,2.403363e+06,3.949029,0.000420,4.919169,2.801289
...,...,...,...,...,...,...,...,...,...,...
1755,2021-02-11,1,DK-81,73.558470,3.624393e+06,2.475768e+06,-6.216205,0.000383,1.495042,4.113037
1756,2021-02-12,1,DK-81,74.618363,4.379149e+06,2.491939e+06,-6.035219,0.000006,1.992372,1.915713
1757,2021-02-13,1,DK-81,76.532522,4.910543e+06,2.494230e+06,-4.408170,0.000000,2.279176,1.357024
1758,2021-02-14,1,DK-81,74.459283,4.752374e+06,2.484782e+06,-3.379998,0.000000,2.772693,2.861502


In [12]:
# This list contains the name of the variables we want to use as predictors
Xs = ['RelativeHumiditySurface', 'SolarRadiation', 'Surfacepressure', 'TemperatureAboveGround',
             'Totalprecipitation', 'UVIndex', 'WindSpeed']

significance_threshold = 0.05

# We start from linear correlation. We print the name of the variable, the correlation
# coefficient and whether the correlation is significant.
for var in Xs:
    corr, pvalue = pearsonr(df["hospitalized_addition"], df[var])
    print(f"{var}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold}\n")

RelativeHumiditySurface
0.213	1.5555940968133262e-19	True

SolarRadiation
-0.328	2.6842293273490432e-45	True

Surfacepressure
-0.036	0.13192932013855818	False

TemperatureAboveGround
-0.388	1.8421932008690805e-64	True

Totalprecipitation
0.020	0.4087068453703637	False

UVIndex
-0.418	1.537805880332672e-75	True

WindSpeed
0.037	0.11981573942373064	False



In [14]:
# Now we do the same for the Spearman rank correlation
for var in Xs:
    corr, pvalue = spearmanr(df["hospitalized_addition"], df[var])
    print(f"{var}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold}\n")

RelativeHumiditySurface
0.295	9.41696780258754e-37	True

SolarRadiation
-0.507	1.0422712265565598e-115	True

Surfacepressure
0.023	0.3361348582493121	False

TemperatureAboveGround
-0.589	1.116912821679257e-164	True

Totalprecipitation
0.011	0.6487617370604414	False

UVIndex
-0.666	9.598316772732291e-226	True

WindSpeed
0.073	0.0021929360954572085	True



In [15]:
# And a third time, for a log-transformed Pearson correlation, note the plus one,
# because you can't take the log of zero
for var in Xs:
    corr, pvalue = pearsonr(np.log(df["hospitalized_addition"] + 1), df[var])
    print(f"{var}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold}\n")


RelativeHumiditySurface
0.258	3.4511079816173335e-28	True

SolarRadiation
-0.459	1.2554270741525784e-92	True

Surfacepressure
0.002	0.9405622280596322	False

TemperatureAboveGround
-0.555	4.6516166179934825e-143	True

Totalprecipitation
-0.036	0.12654233015728236	False

UVIndex
-0.626	2.074355631543586e-192	True

WindSpeed
0.068	0.004377691883576488	True



In [19]:
# Since we are running a lot of tests, we need to correct for multiple hypothesis testing
# This is the Bonferroni correction. If we run N tests and we determine our significance
# threshold as a p-value of 0.005, then we need to make sure that the p-values we look at
# are lower than 0.005 / N. Here I multiply the length of Xs by three, because we are
# running three correlations per variable (linear, Spearman, and log).
significance_threshold_bonf = 0.05 / (len(Xs) * 3)

# Linear
print("Linear")
for var in Xs:
    corr, pvalue = pearsonr(df["hospitalized_addition"], df[var])
    print(f"{var}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold_bonf}\n")

# Spearman
print("Spearman")
for var in Xs:
    corr, pvalue = spearmanr(df["hospitalized_addition"], df[var])
    print(f"{var}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold_bonf}\n")

# Log
print("Log")
for var in Xs:
    corr, pvalue = pearsonr(np.log(df["hospitalized_addition"] + 1), df[var])
    print(f"{var}\n{corr:.3f}\t{pvalue}\t{pvalue < significance_threshold_bonf}\n")

Linear
RelativeHumiditySurface
0.213	1.5555940968133262e-19	True

SolarRadiation
-0.328	2.6842293273490432e-45	True

Surfacepressure
-0.036	0.13192932013855818	False

TemperatureAboveGround
-0.388	1.8421932008690805e-64	True

Totalprecipitation
0.020	0.4087068453703637	False

UVIndex
-0.418	1.537805880332672e-75	True

WindSpeed
0.037	0.11981573942373064	False

Spearman
RelativeHumiditySurface
0.295	9.41696780258754e-37	True

SolarRadiation
-0.507	1.0422712265565598e-115	True

Surfacepressure
0.023	0.3361348582493121	False

TemperatureAboveGround
-0.589	1.116912821679257e-164	True

Totalprecipitation
0.011	0.6487617370604414	False

UVIndex
-0.666	9.598316772732291e-226	True

WindSpeed
0.073	0.0021929360954572085	True

Log
RelativeHumiditySurface
0.258	3.4511079816173335e-28	True

SolarRadiation
-0.459	1.2554270741525784e-92	True

Surfacepressure
0.002	0.9405622280596322	False

TemperatureAboveGround
-0.555	4.6516166179934825e-143	True

Totalprecipitation
-0.036	0.12654233015728236	False

In [20]:
# Let's store all our p-values in a list

pvalues = []
tests = ("linear", "spearman", "log")

for var in Xs:
    corr, pvalue = pearsonr(df["hospitalized_addition"], df[var])
    pvalues.append(pvalue)

for var in Xs:
    corr, pvalue = spearmanr(df["hospitalized_addition"], df[var])
    pvalues.append(pvalue)

for var in Xs:
    corr, pvalue = pearsonr(np.log(df["hospitalized_addition"] + 1), df[var])
    pvalues.append(pvalue)

In [22]:
# We run the Holm-Bonferroni with our usual alpha = 0.005

significant, pholmcorrected, _, _ = multipletests(pvalues, alpha = 0.05, method = "holm")

for i in range(len(significant)):
    print(f"{tests[i // len(Xs)]}\t{Xs[i % len(Xs)]}\t{significant[i]}")

linear	RelativeHumiditySurface	True
linear	SolarRadiation	True
linear	Surfacepressure	False
linear	TemperatureAboveGround	True
linear	Totalprecipitation	False
linear	UVIndex	True
linear	WindSpeed	False
spearman	RelativeHumiditySurface	True
spearman	SolarRadiation	True
spearman	Surfacepressure	False
spearman	TemperatureAboveGround	True
spearman	Totalprecipitation	False
spearman	UVIndex	True
spearman	WindSpeed	True
log	RelativeHumiditySurface	True
log	SolarRadiation	True
log	Surfacepressure	False
log	TemperatureAboveGround	True
log	Totalprecipitation	False
log	UVIndex	True
log	WindSpeed	True
